### Spectral Modulation for Multi-Spectral Images
This function code performs two things based on-
- Level 2 corrected images of the original dataset.
- Modulation based classification for multi-spectral images.

Idea for the Spectral Modulation code:
- Suppose there are 4 bands- b1,b2,b3,b4 and pattern as (120,110,100,90) then modulation pattern is:'222222' based on (120>110,120>100,120>90,110>100,110>90,100>90)
- '2' represents preceeding band value is greater
- '1' represents succeeding band value is greater
- '0' represents both band values are equal
- Each pixel vector of size 4 is identified in underlying classes. 

In [1]:
!apt-get update
#Step 2
!apt-get install libgdal-dev -y
#Step 3
!apt-get install python-gdal -y
#Step 4
!apt-get install python-numpy python-scipy -y
#Step 5

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Get:12 https://developer.downl

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import gdal
import osr
import os

This function reads the tif file on a given path and returns its data , projection, geotransform and targetprj.

## Input:
- band(str) : Name of the band.
- PATH(str): path of the file.

In [3]:
def readBand(band,PATH):
    img = gdal.Open(PATH)
    data = np.array(img.GetRasterBand(1).ReadAsArray())
    spatialRef = img.GetProjection()
    geoTransform = img.GetGeoTransform()
    targetprj = osr.SpatialReference(wkt = img.GetProjection())
    
    return data, spatialRef, geoTransform, targetprj


This function converts np array to a raster file and store in tif format.
## Input:
- array: numpy array containing DN values
-geoTransform: affine transformation coefficients
- projection: projection info
- filename: output path

In [4]:
def writeBand(array, geoTransform, projection, filename, classified=False, dtype=gdal.GDT_UInt16):
  
    pixels_x = array.shape[1]
    pixels_y = array.shape[0]
    
    driver = gdal.GetDriverByName('GTiff')
    dataset = driver.Create(
            filename,
            pixels_x,
            pixels_y,
            1,
            dtype)
    dataset.SetGeoTransform(geoTransform)
    dataset.SetProjection(projection)
    
    band = dataset.GetRasterBand(1)
    if classified:
        colors = gdal.ColorTable()
        for class_value in classified:
            colors.SetColorEntry(class_value[0], tuple(np.random.choice(range(256), size=3)))
        band.SetRasterColorTable(colors)
        band.SetRasterColorInterpretation(gdal.GCI_PaletteIndex)
        
    band.SetNoDataValue(0)
    band.WriteArray(array)
    dataset.FlushCache()

This is spectral modulation function for feature band classification
## Input:
- bands: Tuple of bands on which spectral modulation is to be performed.
-outpath: output path

In [5]:
def spectral_modulation(bands,outpath):
  rows,cols=bands[0][0].shape
  classified_image=np.zeros((rows,cols),dtype=np.float16)
  water_image=np.zeros((rows,cols),dtype=np.float16)
  classes={}
  class_count=1

  for row in range(rows):
    for col in range(cols):
      spectra=[]
      for band in bands:
        spectra.append(band[0][row][col])
      
      modulation=""

      for i in range(len(spectra)):
        for j in range(i+1,len(spectra)):
          if spectra[i]<spectra[j]:
            modulation+='2'
          elif spectra[i]>spectra[j]:
            modulation+='1'
          else:
            modulation+='0'
      
      if modulation in classes:
        classified_image[row][col]=classes[modulation][0]
        classes[modulation][1]+=1
      else:
        classified_image[row][col]=class_count
        classes[modulation]=[class_count,0]
        class_count+=1

      if  (modulation=='111222') or (modulation=='221011') or (modulation=='111111'):
        water_image[row][col] = 0
      else:
        water_image[row][col] = 255

  print(classes)
  print(water_image)
  print(classified_image)
  writeBand(classified_image, bands[0][2], bands[0][1], '/content/drive/MyDrive/SPEC MOD/LS8_TM_all_classes__' +outpath+ '.tif', classified=tuple(classes.values()))
  writeBand(water_image, bands[0][2], bands[0][1], '/content/drive/MyDrive/SPEC MOD/LS8_TM_swir_water__' + outpath + '.tif')
